##### Author：马肖
##### E-mail：maxiaoscut@aliyun.com
##### Github：https://github.com/Albertsr

In [1]:
import time
import numpy as np
import pandas as pd

import mahal_dist as md
import Robust_PCC as rp
import Recon_Error_PCA as rep
import Recon_Error_KPCA as rek

from sklearn.datasets import *
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

In [2]:
def predict_anomaly_indices(X, contamination):
    
    # 孤立森林
    iforest = IsolationForest(n_estimators=125, contamination=contamination, behaviour='new', random_state=2018, n_jobs=-1)
    iforest_result = iforest.fit_predict(X)
    anomaly_num = len(np.where(iforest_result==-1)[0])
    # 分数越小于0，越有可能是异常值
    anomaly_score = iforest.decision_function(X)
    if_idx = np.argsort(anomaly_score)[:anomaly_num]
    
    # LOF
    lof = LocalOutlierFactor(contamination=contamination, p=2, novelty=False, n_jobs=-1)
    lof.fit(X)
    score = -lof.negative_outlier_factor_ 
    lof_idx = np.argsort(-score)[:anomaly_num]
    
    # RobustPCC
    rpcc = rp.RobustPCC(X, X, gamma=0.01, quantile=99)
    rpcc_idx = rpcc.test_anomaly_idx()[:anomaly_num]
    
    # 马氏距离
    dist = md.mahal_dist(X)
    md_idx = np.argsort(-dist)[:anomaly_num]
    
    # LinearPCA重构
    pre = rep.PCA_Recon_Error(X, contamination=contamination)
    pre_idx = pre.anomaly_idx()
    
    # KernelPCA重构
    kre = rek.KPCA_Recon_Error(X, contamination=contamination)
    kre_idx = kre.anomaly_idx()
    
    anomaly_indices = [if_idx, lof_idx, rpcc_idx, md_idx, kre_idx, pre_idx]
    return np.array(anomaly_indices)

#### 对比

In [3]:
def anomaly_indices_contrast(X, contamination=0.02, observed_anomaly_indices=None):
    start = time.time()
    anomaly_indices = predict_anomaly_indices(X, contamination)
    if observed_anomaly_indices:
        baseline = observed_anomaly_indices  
    else: 
        # 如果异常样本的索引未知，则以孤立森林判定的异常索引为Baseline
        baseline = anomaly_indices[0]
    
    indices_contrast = pd.DataFrame(anomaly_indices)
    algorithms = ['Isolation Forest', 'LOF', 'Robust PCC', 'Mahalanobis Dist', 'KPCA_Recon_Error', 'PCA_Recon_Error']
    indices_contrast.index = algorithms
    indices_contrast.index.name = 'Algorithm'

    # 统计各算法预测出的异常索引与baseline的相交个数
    def indices_intersect(indices_predicted):
        return sum(np.isin(indices_predicted, baseline))

    # 在indices_contrast中新增一列'Baseline_Same'，用于存放各算法预测与baseline的相交个数
    indices_contrast['Baseline_Same'] = list(map(indices_intersect, anomaly_indices))
    # 根据Baseline_Same的取值大小，对indices_contrast各行进行降序排列
    indices_contrast.sort_values(by=['Baseline_Same'], ascending=False, inplace=True)
    print('Dataset_Shape:{:}, Running_Time:{:.2f}s'.format(X.shape, (time.time()-start)))
    return indices_contrast

In [4]:
boston = load_boston().data
anomaly_indices_contrast(boston)

Dataset_Shape:(506, 13), Running_Time:7.09s


,0,1,2,3,4,5,6,7,8,9,10,Baseline_Same
Algorithm,,,,,,,,,,,,
Isolation Forest,155,283,414,410,142,418,145,152,156,412,364,11
KPCA_Recon_Error,380,418,405,410,414,283,364,155,365,142,427,7
Robust PCC,380,418,405,410,414,427,404,398,413,490,142,4
Mahalanobis Dist,380,418,405,410,365,155,490,367,492,364,491,4
PCA_Recon_Error,380,418,405,410,365,367,155,214,368,490,492,3
LOF,380,488,491,492,405,25,27,102,155,156,18,2


In [5]:
cancer = load_breast_cancer().data
anomaly_indices_contrast(cancer)

Dataset_Shape:(569, 30), Running_Time:4.76s


,0,1,2,3,4,5,6,7,8,9,10,11,Baseline_Same
Algorithm,,,,,,,,,,,,,
Isolation Forest,122,461,212,108,152,78,82,3,352,12,567,0,12
KPCA_Recon_Error,212,461,152,122,3,68,78,213,192,9,190,108,7
Robust PCC,212,461,12,265,122,152,190,368,290,38,78,68,6
Mahalanobis Dist,152,212,461,12,68,290,213,71,192,122,38,78,6
PCA_Recon_Error,152,212,461,68,290,213,192,71,122,12,190,78,6
LOF,461,212,38,265,101,31,180,417,122,368,352,5,4


In [6]:
def generate_dataset(seed):
    rdg = np.random.RandomState(seed)
    row = rdg.randint(2500, 3000)
    col = rdg.randint(30, 35)
    contamination = rdg.uniform(0.015, 0.025)
    
    outlier_num = int(row*contamination)
    inlier_num = row - outlier_num
    
    # 正常样本集服从标准正态分布
    inliers = rdg.randn(inlier_num, col)
    
    # 如果outlier_num为奇数，row_1=outlier_num//2，否则row_1=int(outlier_num/2)
    row_1 = outlier_num//2 if np.mod(outlier_num, 2) else int(outlier_num/2)
    row_2 = outlier_num - row_1
    
    # outliers_sub_1服从伽玛分布；outliers_sub_2服从指数分布
    outliers_sub_1 = rdg.gamma(shape=2, scale=0.5, size=(row_1 , col))
    outliers_sub_2 = rdg.exponential(1.5, size=(row_2, col))
    outliers = np.r_[outliers_sub_1, outliers_sub_2]
    
    # 将inliers与outliers在axis=0方向上予以整合，构成实验数据集
    dataset = np.r_[inliers, outliers]
    outliers_indices = range(len(dataset))[inlier_num:]
    return dataset, contamination, outliers_indices

def return_algo(seed):
    dataset, contamination, outliers_indices = generate_dataset(seed)
    result = anomaly_indices_contrast(dataset, contamination, outliers_indices)
    return result.index

In [7]:
seeds = np.random.RandomState(2018).choice(range(1000), size=10, replace=False)
indices_sorted = list(map(return_algo, seeds))
index = ['Dataset_' + str(i) for i in range(len(seeds))]
algo_sorted = pd.DataFrame(indices_sorted, index=index)
algo_sorted.index.name = 'VerifyData'

Dataset_Shape:(2863, 34), Running_Time:152.63s
Dataset_Shape:(2933, 33), Running_Time:151.86s
Dataset_Shape:(2613, 34), Running_Time:119.28s
Dataset_Shape:(2776, 34), Running_Time:138.40s
Dataset_Shape:(2671, 30), Running_Time:121.92s
Dataset_Shape:(2664, 34), Running_Time:133.51s
Dataset_Shape:(2896, 34), Running_Time:151.37s
Dataset_Shape:(2793, 34), Running_Time:137.62s
Dataset_Shape:(2978, 31), Running_Time:146.64s
Dataset_Shape:(2797, 30), Running_Time:119.97s


In [8]:
sorted_algo = algo_sorted.copy()
mode = sorted_algo.mode(axis=0)
mode

,0,1,2,3,4,5
0,Robust PCC,Isolation Forest,KPCA_Recon_Error,Mahalanobis Dist,LOF,PCA_Recon_Error
1,NaN,NaN,Mahalanobis Dist,NaN,NaN,NaN


In [9]:
def revise_mode(mode):
    target_idx = mode.notnull().sum().idxmax()
    target_col = mode.iloc[:, target_idx]
    
    # 去掉first_row中在target_idx索引处的值，成为first_row_trimmed
    first_row = mode.iloc[0, :] 
    cond = np.isin(first_row.index, target_idx, invert=True)
    first_row_trimmed = first_row[cond]
    
    # target_col内元素不在first_row_trimmed之内，则保留，否则删除
    cond = np.isin(target_col, first_row_trimmed, invert=True)
    target_idx_mode = target_col[cond].values[0]
    
    first_row[target_idx] = target_idx_mode
    return first_row.values

In [10]:
if len(mode) == 1:
    sorted_algo.loc['Mode(众数)'] = mode.values.ravel()
else:
    sorted_algo.loc['Mode(众数)'] = revise_mode(mode)
    
#columns = ['Baseline','First Algorithm', 'Second Algorithm', 'Thrid Algorithm', 'Fourth Algorithm', 'Fifth Algorithm']
columns = ['Algorithm 1st', 'Algorithm 2nd', 'Algorithm 3rd', 'Algorithm 4th', 'Algorithm 5th', 'Algorithm 6th']
sorted_algo.columns = columns

def show(row):
    color = 'yellow'
    return 'background-color: %s' % color
sorted_algo.style.applymap(show, subset=pd.IndexSlice['Mode(众数)':, :])

,Algorithm 1st,Algorithm 2nd,Algorithm 3rd,Algorithm 4th,Algorithm 5th,Algorithm 6th
VerifyData,,,,,,
Dataset_0,Robust PCC,Isolation Forest,LOF,Mahalanobis Dist,KPCA_Recon_Error,PCA_Recon_Error
Dataset_1,Robust PCC,Mahalanobis Dist,KPCA_Recon_Error,LOF,Isolation Forest,PCA_Recon_Error
Dataset_2,Robust PCC,KPCA_Recon_Error,Isolation Forest,Mahalanobis Dist,LOF,PCA_Recon_Error
Dataset_3,Robust PCC,Isolation Forest,KPCA_Recon_Error,Mahalanobis Dist,LOF,PCA_Recon_Error
Dataset_4,Robust PCC,Isolation Forest,Mahalanobis Dist,KPCA_Recon_Error,LOF,PCA_Recon_Error
Dataset_5,Robust PCC,Isolation Forest,LOF,Mahalanobis Dist,KPCA_Recon_Error,PCA_Recon_Error
Dataset_6,Robust PCC,KPCA_Recon_Error,Isolation Forest,LOF,Mahalanobis Dist,PCA_Recon_Error
Dataset_7,Robust PCC,Isolation Forest,Mahalanobis Dist,KPCA_Recon_Error,LOF,PCA_Recon_Error
Dataset_8,Robust PCC,Isolation Forest,KPCA_Recon_Error,LOF,Mahalanobis Dist,PCA_Recon_Error
